# Find Correct Employer-Side Contributions
For the calculation of a pension cost for each agency, we need the yearly employer-side contributions to the Masschusetts Statewide pension fund.
There are two places a plausable measure of this yearly contribution show up in cthru:

- object class = "(DD) PENSION & INSURANCE RELATED EX"
- appropriation_name = "(06121020) STATE RETIREMENT BD COMMONWEALTH PENSION"

The appropration name expenditures come out to 1.9ish billion per year, which is close to the number I can find the state budget [here](https://budget.digital.mass.gov/summary/fy20/enacted/independents/treasurer/treasurer?tab=budget-tracking) (look for line 0612-1020). 

It is probably true that the object_class expenditures include the teachers's retirement fund, but contributions to that fund total around 2.6 billion per year, so it's more money than both  

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sodapy import Socrata
app_token = "2Qa1WiG8G4kj1vGVd2noK7zP0"
client = Socrata("cthru.data.socrata.com", app_token)
client.timeout = 40

In [4]:
import pandas as pd
pd.set_option("display.max.columns",100)

In [11]:
big_pensions = pd.DataFrame(client.get("pegc-naaa", where = "object_class = '(DD) PENSION & INSURANCE RELATED EX' AND budget_fiscal_year = 2020",  limit=9999999))

In [14]:
big_pensions["amount"] = big_pensions["amount"].astype(float)

In [17]:
big_pensions.groupby("object_code").sum()["amount"].sort_values(ascending=False)

object_code
(D04) RETIREMENT ALLOWANCE-TEACHERS                                 3.290207e+09
(D06) EMPLOYEE HEALTH & LIFE INSURANCE                              2.745379e+09
(D05) RETIREMENT ALLOWANCE-STATE EMPLOYEES                          2.448302e+09
(D09) FRINGE BENEFIT COST RECOUPMENT                                2.117431e+08
(D08) HEALTH & WELFARE TRUST FUND                                   6.950973e+07
(D01) RETIREMENT ALLOWANCES                                         3.618364e+07
(D90) OPERATING TRANSFER                                            2.586526e+07
(D12) UNEMPLOYMENT COMPENSATION INSURANCE PREMIUM PAYMENT           1.891380e+07
(D20) PENSION & INSURANCE RELATED EXPENDITURES                      1.456065e+07
(D10) SURETY OF EMPLOYEES                                           8.900861e+06
(D19) UNIVERSAL HEALTH INSURANCE PAYMENTS                           5.902620e+06
(D24) WORKERS' COMPENSATION  OR (IPP) PAYMENT                       5.053151e+06
(D15) WORKERS' C

In [19]:
D05_code = big_pensions[big_pensions["object_code"].str.contains("(D05)")]

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [25]:
D05_code["amount"].sum()

2448301953.73

Ok this makes sense. This is payouts from pension fund per fiscal year, and the number here roughly matches the numbers at 
http://cthrupensions.mass.gov/#!/year/2020//pay1/explore/0-0-0/retirement_system

So what we should be using is the appropriation name because we are looking for is employer-side contributions. This all makes sense I feel good about this

Another important note: the state retirement benefits system seems to have complete data for 2020. This means that the "year" it explains is fiscal year, otherwise the numbers it shows would be incomplete.
Payroll, however, is by calendar year as the 2020 number at this point is lower. 